In [1]:
from pyspark.sql import functions as f

In [2]:
countryCode = 'nl'

#### transform new OSM data

In [4]:
list_columns = ['operatorID','osmId','placeIdGoogle','ohubId',\
                            'name','address','postalCode','city','latitude','longitude','businessType','website','phone',\
                            'nameGoogle','addressGoogle','postalCodeGoogle','cityGoogle',\
                            'latitudeGoogle','longitudeGoogle','businessTypeGoogle','websiteGoogle',
                            'type','uid','nameOSM','addressOSM','postalCodeOSM','cityOSM',\
                            'latitudeOSM','longitudeOSM','businessTypeOSM','websiteOSM','cuisineTypeOSM',\
                            'nameOHUB','addressOHUB','postalcodeOhub','cityOHUB','concatID']

In [5]:
nl_osm_new = spark.table('dev_sources_osm.cleaned_total_operator_base').where(f.lower(f.col('countryCode'))== countryCode) \
                          .where(f.col('street').isNotNull()) \
                          .where(f.col('name').isNotNull()) \
                          .withColumn('operatorID', f.concat(f.col('osmId'),f.lit(' /  / '))) \
                          .withColumn('osmId',f.col('osmid').cast('string')) \
                          .withColumn('placeIdGoogle',f.lit(None).cast('string')) \
                          .withColumn('name',f.col('name')) \
                          .withColumn('address',(f.concat(f.when(f.col('street').isNotNull(),f.col('street')).otherwise(f.lit('')),f.lit(" "), \
                                                          f.when(f.col('housenumber').isNotNull(),f.col('housenumber')).otherwise(f.lit(''))))) \
                          .withColumn('postalCode',f.regexp_replace(f.when(f.col('postcode').isNotNull(),f.col('postcode')).otherwise(f.lit('')),'[^a-zA-Z0-9]','').cast('string')) \
                          .withColumn('city',f.col('city')) \
                          .withColumn('latitude',f.col('latitude').cast('float')) \
                          .withColumn('longitude',f.col('longitude').cast('float')) \
                          .withColumn('businessType',f.col('BusinessTypesList').cast('string')) \
                          .withColumn('website',f.col('website')) \
                          .withColumn('phone',f.col('phone')) \
                          .withColumn('nameGoogle',f.lit(None).cast('string')) \
                          .withColumn('addressGoogle',f.lit(None).cast('string')) \
                          .withColumn('postalCodeGoogle',f.lit(None).cast('string')) \
                          .withColumn('cityGoogle',f.lit(None).cast('string')) \
                          .withColumn('latitudeGoogle',f.lit(None).cast('string')) \
                          .withColumn('longitudeGoogle',f.lit(None).cast('string')) \
                          .withColumn('businessTypeGoogle',f.lit(None).cast('string')) \
                          .withColumn('websiteGoogle',f.lit(None).cast('string')) \
                          .withColumn('phoneGoogle',f.lit(None).cast('string')) \
                          .withColumn('ohubId',f.lit(None).cast('string'))\
                          .withColumn('nameOHUB',f.lit(None).cast('string'))\
                          .withColumn('addressOHUB',f.lit(None).cast('string'))\
                          .withColumn('postalcodeOhub',f.lit(None).cast('string'))\
                          .withColumn('cityOHUB',f.lit(None).cast('string'))\
                          .withColumn('concatID',f.lit(None).cast('string'))\
                          .withColumn('channelOHUB',f.lit(None).cast('string')) \
                          .select('operatorID',
                                  'osmId',
                                  'placeIdGoogle',
                                  'ohubid',
                                  'name',
                                  'address',
                                  'postalCode',
                                  'city',
                                  'latitude',
                                  'longitude',
                                  'businessType',
                                  'website',
                                  'phone',
                                  'nameGoogle',
                                  'addressGoogle',
                                  'postalCodeGoogle',
                                  'cityGoogle',
                                  'latitudeGoogle',
                                  'longitudeGoogle',
                                  'businessTypeGoogle',
                                  'websiteGoogle',
                                  'type',                                                            
                                  'uid',
                                  f.col('name').alias('nameOSM'),
                                  f.col('address').alias('addressOSM'),
                                  f.col('postalCode').alias('postalCodeOSM'),
                                  f.col('city').alias('cityOSM'),
                                  f.col('latitude').alias('latitudeOSM'),
                                  f.col('longitude').alias('longitudeOSM'),
                                  f.col('businessType').alias('businessTypeOSM'),
                                  f.col('website').alias('websiteOSM'),
                                  f.col('cuisineCleanList').cast('string').alias('cuisineTypeOSM'),
                                  'nameOHUB','addressOHUB','postalcodeOhub','cityOHUB','concatID')

In [6]:
nl_enriched = spark.table('dev_derived_ouniverse.output_total_universe_nl_enriched')
osm_id_list_in_universe = [str(row.osmid) for row in nl_enriched.where(f.col('osmid').isNotNull()).select("osmid").collect()]

print(len(osm_id_list_in_universe))

36152

In [7]:
nl_osm_new = nl_osm_new.select(list_columns)
for col in nl_enriched.columns:
  if col not in nl_osm_new.columns:
    nl_osm_new = nl_osm_new.withColumn(col,f.lit(None).cast(nl_enriched.schema[col].dataType))
    
nl_osm_new = nl_osm_new.select(nl_enriched.columns)

### Check OSM part in Universe and OSM in current OSM data

In [9]:
checkinUniverse = nl_enriched.where(f.col('osmid').isNotNull()).join(nl_osm_new, on = 'osmId', how = 'left_anti')
checkinUniverse.count()

Out[15]: 13693

In [10]:
checkinOSM= nl_osm_new.join(nl_enriched.where(f.col('osmid').isNotNull()), on = 'osmId', how = 'left_anti')
checkinOSM.count()

Out[9]: 15108

In [11]:
display(nl_osm_new.where(f.col('name') == "Grand Cafe 't Elfde Gebod"))

operatorId,PlaceIDGoogle,osmId,ohubID,name,address,postalCode,city,Latitude,Longitude,businessType,Website,cuisineType,NameGoogle,AddressGoogle,PostalCodeGoogle,CityGoogle,LatitudeGoogle,LongitudeGoogle,businessTypeGoogle,WebsiteGoogle,type,uid,nameOSM,AddressOSM,PostalcodeOSM,cityOSM,LatitudeOSM,LongitudeOSM,cuisineTypeOSM,websiteOSM,concatID,nameOhub,postalcodeOhub,addressOhub,cityOhub,source,globalChannel,chain,buying,valueTier,valueTierDescription,d_buyer,id,keyword,uniqueProductCount,SAP_DEX_ID,is_visited,SAPID,cp_mobileNumber,phone,countryGoogle,InternationalPhoneNumberGoogle,uniqueID,take_away,isTargetUniverse,menuHasTerras,primary_topcategory,businessTypeOSM,channelOhub,menuDishes,globalListChannels,operatorLifeCycle_Stage,SALESREP


In [12]:
display(checkinUniverse)
# https://www.openstreetmap.org/node/2750544386
# Flora Boskoop


osmId,operatorId,PlaceIDGoogle,ohubID,name,address,postalCode,city,Latitude,Longitude,businessType,Website,cuisineType,NameGoogle,AddressGoogle,PostalCodeGoogle,CityGoogle,LatitudeGoogle,LongitudeGoogle,businessTypeGoogle,WebsiteGoogle,type,uid,nameOSM,AddressOSM,PostalcodeOSM,cityOSM,LatitudeOSM,LongitudeOSM,cuisineTypeOSM,websiteOSM,concatID,nameOhub,postalcodeOhub,addressOhub,cityOhub,source,globalChannel,chain,buying,valueTier,valueTierDescription,d_buyer,id,keyword,uniqueProductCount,SAP_DEX_ID,is_visited,SAPID,cp_mobileNumber,phone,countryGoogle,InternationalPhoneNumberGoogle,uniqueID,take_away,isTargetUniverse,menuHasTerras,primary_topcategory,businessTypeOSM,channelOhub,menuDishes,globalListChannels,operatorLifeCycle_Stage,SALESREP
1085657792,1085657792 / /,null,null,Grand Cafe 't Elfde Gebod,Koningin Wilhelmina Boulevard,2202GV,Noordwijk,52.247893000000005,4.4333085,pub,http://www.hotelvanoranje.nl/,Unknown,null,null,null,null,null,null,null,null,node,0,Grand Cafe 't Elfde Gebod,null,null,null,52.247893000000005,4.4333085,null,http://www.hotelvanoranje.nl/,null,null,null,null,null,OSM,Cafe,Unknown,null,10,null,0,null,null,null,null,null,null,null,null,null,,null,false,1,true,null,pub,null,tonijn,"[Other, Pub, Cafe]",E1B0-low,Ron Quartel
1288186491,1288186491 / /,null,null,OBS Commissaris Gaarlandt,Rembrandtstraat,7948AT,Meppel,52.733274200000004,6.1666862,school,null,Unknown,null,null,null,null,null,null,null,null,node,0,OBS Commissaris Gaarlandt,null,null,null,52.733274200000004,6.1666862,null,null,null,null,null,null,null,OSM,Schools - Unknown level,Unknown,null,9,€0.0 - €36.0 (average value = 20.0),0,null,null,null,null,null,null,null,null,null,,null,false,1,null,null,school,null,null,"[Other, Schools - Unknown level]",E1B0-low,Geartsje Stegenga
1345799233,1345799233 / /,null,null,Bodega Bora Bora,Markt,5554CD,Valkenswaard,51.350611400000005,5.4585947,bar,null,Unknown,null,null,null,null,null,null,null,null,node,0,Bodega Bora Bora,null,null,null,51.350611400000005,5.4585947,null,null,null,null,null,null,null,OSM,Other,Unknown,null,10,null,0,null,null,null,null,null,null,null,null,null,,null,false,0,null,null,bar,null,null,[Other],E0B0-low,Johan van Wageningen
1392117337,1392117337 / /,null,null,Paalkampeerterrein Klinkenbelt,Klinkenbeltweg,7448SC,null,52.372447900000004,6.4284192000000004,camp_site,null,Unknown,null,null,null,null,null,null,null,null,node,0,Paalkampeerterrein Klinkenbelt,null,null,null,52.372447900000004,6.4284192000000004,null,null,null,null,null,null,null,OSM,Campground,Unknown,null,10,null,0,null,null,null,null,null,null,null,null,null,,null,false,0,null,null,camp_site,null,null,"[Other, Campground]",E0B0-low,Danny van de Put
1476416995,1476416995 / /,null,null,Aloysius,Meester Postlaan,3155BM,Midden-Delfland,51.9340344,4.2729796,school,null,Unknown,null,null,null,null,null,null,null,null,node,0,Aloysius,null,null,null,51.9340344,4.2729796,null,null,null,null,null,null,null,OSM,Schools - Unknown level,Unknown,null,10,null,0,null,null,null,null,null,null,null,null,null,,null,false,1,null,null,school,null,null,"[Other, Schools - Unknown level]",E1B0-low,Marc Bontekoe
1547889731,1547889731 / /,null,null,Polderstraat,Polderstraat,2952AK,null,51.860169600000006,4.6611454000000005,fuel,null,Unknown,null,null,null,null,null,null,null,null,node,0,Polderstraat,null,null,null,51.860169600000006,4.6611454000000005,null,null,null,null,null,null,null,OSM,Other,Unknown,null,9,€0.0 - €36.0 (average value = 20.0),0,null,null,null,null,null,null,null,null,null,,null,false,0,null,null,fuel,null,null,[Other],E0B0-low,Marc Bontekoe
1551002291,1551002291 / /,null,null,XL&t,Zwanenburgseweg,4384LW,Vlissingen,51.4556807,3.5496652,restaurant,https://www.tenbzeeland.nl/restaurant,Unknown,null,null,null,null,null,null,null,null,node,0,XL&t,null,null,null,51.4556807,3.5496652,null,https://www.tenbzeeland.nl/restaurant,null,null,null,null,null,OSM,Other Restaurant,Unknown,null,10,null,0,null,null,null,null,null,null,null,nu

#### update OSM in Universe

In [14]:
nl_osm_new.count()

Out[9]: 37567

In [15]:
nl_osm_new_not_in_universe = nl_osm_new.where(~f.col('osmid').isin(osm_id_list_in_universe))

In [16]:
display(checkinOSM.where(f.col('osmId').contains('restaurant')))

osmId,operatorId,PlaceIDGoogle,ohubID,name,address,postalCode,city,Latitude,Longitude,businessType,Website,cuisineType,NameGoogle,AddressGoogle,PostalCodeGoogle,CityGoogle,LatitudeGoogle,LongitudeGoogle,businessTypeGoogle,WebsiteGoogle,type,uid,nameOSM,AddressOSM,PostalcodeOSM,cityOSM,LatitudeOSM,LongitudeOSM,cuisineTypeOSM,websiteOSM,concatID,nameOhub,postalcodeOhub,addressOhub,cityOhub,source,globalChannel,chain,buying,valueTier,valueTierDescription,d_buyer,id,keyword,uniqueProductCount,SAP_DEX_ID,is_visited,SAPID,cp_mobileNumber,phone,countryGoogle,InternationalPhoneNumberGoogle,uniqueID,take_away,isTargetUniverse,menuHasTerras,primary_topcategory,businessTypeOSM,channelOhub,menuDishes,globalListChannels,operatorLifeCycle_Stage,SALESREP
95288389,null,null,null,Meram,Pretoriusstraat 22,,Amsterdam,null,null,[restaurant],null,null,null,null,null,null,null,null,null,null,node,0,Meram,null,null,Amsterdam,null,null,[turkish],https://meram.nl/,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,[restaurant],null,null,null,null,null
251177434,null,null,null,Wing Kee,Zeedijk 78,1012CR,Amsterdam,null,null,[restaurant],null,null,null,null,null,null,null,null,null,null,node,0,Wing Kee,null,null,Amsterdam,null,null,[chinese],null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,[restaurant],null,null,null,null,null
303274482,null,null,null,The Mart,High Tech Campus 1,5656AE,Eindhoven,null,null,[restaurant],null,null,null,null,null,null,null,null,null,null,node,0,The Mart,null,null,Eindhoven,null,null,null,https://www.hightechcampus.com/facilities/the-mart,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,[restaurant],null,null,null,null,null
303274485,null,null,null,Alfresco,High Tech Campus 1,5656AE,Eindhoven,null,null,[restaurant],null,null,null,null,null,null,null,null,null,null,node,0,Alfresco,null,null,Eindhoven,null,null,null,https://www.hightechcampus.com/facilities/alfresco/,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,[restaurant],null,null,null,null,null
303274488,null,null,null,The Lounge,High Tech Campus 1,5656AE,Eindhoven,null,null,[restaurant],null,null,null,null,null,null,null,null,null,null,node,0,The Lounge,null,null,Eindhoven,null,null,null,https://www.hightechcampus.com/facilities/the-lounge,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,[restaurant],null,null,null,null,null
303797681,null,null,null,Nieuw Schaijk,Rijksweg 46,5374RB,Schaijk,null,null,[restaurant],null,null,null,null,null,null,null,null,null,null,node,0,Nieuw Schaijk,null,null,Schaijk,null,null,null,https://www.nieuwschaijk.nl,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,+31486461294,null,null,null,null,null,null,null,[restaurant],null,null,null,null,null
344917270,null,null,null,De Oude Maas,Ossestraat 11,5367NE,Macharen,null,null,[restaurant],null,null,null,null,null,null,null,null,null,null,node,0,De Oude Maas,null,null,Macharen,null,null,[pancake],https://deoudemaas.nl,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,+31412647890,null,null,null,null,null,null,null,[restaurant],null,null,null,null,null
347010429,null,null,null,De Knip,Kniplaan 22,,Voorschoten,null,null,[restaurant],null,null,null,null,null,null,null,null,null,null,node,0,De Knip,null,null,Voorschoten,null,null,[regional],null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,[restaurant],null,null,null,null,null
349194015,null,null,null,Tong Ah,Panweg 118,3705GG,Zeist,null,null,[res

In [17]:
display(checkinUniverse.where(f.lower(f.col('name')).isin(['joelia', 'garam masala', 'de eenhoorn', 'het stadhuis', 'restaurant osteria'])))

osmId,operatorId,PlaceIDGoogle,ohubID,name,address,postalCode,city,Latitude,Longitude,businessType,Website,cuisineType,NameGoogle,AddressGoogle,PostalCodeGoogle,CityGoogle,LatitudeGoogle,LongitudeGoogle,businessTypeGoogle,WebsiteGoogle,type,uid,nameOSM,AddressOSM,PostalcodeOSM,cityOSM,LatitudeOSM,LongitudeOSM,cuisineTypeOSM,websiteOSM,concatID,nameOhub,postalcodeOhub,addressOhub,cityOhub,source,globalChannel,chain,buying,valueTier,valueTierDescription,d_buyer,id,keyword,uniqueProductCount,SAP_DEX_ID,is_visited,SAPID,cp_mobileNumber,phone,countryGoogle,InternationalPhoneNumberGoogle,uniqueID,take_away,isTargetUniverse,menuHasTerras,primary_topcategory,businessTypeOSM,channelOhub,menuDishes,globalListChannels,operatorLifeCycle_Stage,SALESREP
471037983,471037983 / /,null,null,Restaurant Osteria,Markt 2a,5492AB,Meierijstad,51.565047400000005,5.4611972,restaurant,null,Unknown,null,null,null,null,null,null,null,null,node,0,Restaurant Osteria,null,null,null,51.565047400000005,5.4611972,null,null,null,null,null,null,null,OSM,Other Restaurant,Unknown,null,10,null,0,null,null,null,null,null,null,null,null,null,,null,false,1,null,null,restaurant,null,null,"[Other, Other Restaurant]",E1B0-low,Johan van Wageningen


In [18]:
print('current count from new osm', nl_osm_new.select('osmId').count(),'\n',
     'current count in universe nl', nl_enriched.where(f.col('osmid').isNotNull()).count(),'\n',
     'differentation', nl_osm_new_not_in_universe.count())

current count from new osm 37567 
 current count in universe nl 36152 
 differentation 15108

In [19]:
final = nl_enriched.union(nl_osm_new_not_in_universe)

#### Check the osm part in new universe

In [21]:
print('osm count after:', final.where(f.col('osmid').isNotNull()).count(), '\n',
     'osm count before', nl_enriched.where(f.col('osmid').isNotNull()).count())

osm count after: 51260 
 osm count before 36152